In [1]:
%matplotlib inline

import numpy as np
import matplotlib as plt

np.set_printoptions(precision=2, suppress=True, formatter={'float': '{: 0.2f}'.format})

In [2]:
MD = 0
INC = 1
AZ = 2
N = 0
E = 1
V = 2

def reflect(u, v):
    """
    Returns the vector v reflected in the unit vector u.
    """
    return 2.0 * np.dot(v, u) * u - v

def angle(u, v):
    """
    Returns the unoriented angle bewteen two non-singular unit length vectors.
    """
    return 2.0 * np.arctan(np.linalg.norm(v - u) / np.linalg.norm(u + v))

def toUnitDir(degINC, degAZ):
    """
    Convert spherical coordinates to cubic coordinates (NEV), a unit length direction vector
    in a right handed coordinate system.
    """
    inc = np.deg2rad(degINC)
    az = np.deg2rad(degAZ)
    deltaN = np.sin(inc) * np.cos(az)
    deltaE = np.sin(inc) * np.sin(az)
    deltaV = np.cos(inc)
    return np.array([deltaN, deltaE, deltaV])

def toSpherical(unit_t):
    """Convert a unit direction vector in NEV coordinates to a tuple of Inclination and Azimuth."""
    inc = np.arctan2(np.sqrt(unit_t[N] * unit_t[N] + unit_t[E] * unit_t[E]), unit_t[V])
    az = np.arctan2(unit_t[E], unit_t[N])
    # return a result for az such that az is [0.0, 360.0)
    if az < 0.0:
        az += 2.0 * np.pi
    return np.array([np.rad2deg(inc), np.rad2deg(az)])

def arc2chord(t1, t2, arclen):
    """
    Calculates the relative vector between two survey stations. Assumes survey values are correct.
    """
    t12 = t1 + t2 # add the tangent vectors; a vector that points to the end of the arc from the start
    lsqrd_t12 = np.dot(t12, t12) # the length squared of the vector sum
    if (1.0 + lsqrd_t12) == 1.0: # test for anti-paralell tangent vectors. Also prevents div-by-zero when unitizing the direction vector
        return np.array([np.nan, np.nan, np.nan]) # np.nan's so any use will signal the sigular case
    len_t12 = np.sqrt(lsqrd_t12) # the length of the addition vector
    t21 = t2 - t1 # subtract the tangent vectors; the chord of on a unit circle
    len_t21 = np.sqrt(np.dot(t21, t21)) # the length of the subtraction vector
    alpha = 2 * np.arctan(len_t21 / len_t12) # the unoriented angle between the tangent vectors; the arc length on a unit circle
    if len_t21 < alpha: # do the circle geometry test
        return arclen * (len_t21 / alpha) * (t12 / len_t12) # arc-2-chord. For robust numeric evaluation the order of operations here are important
    else:
        return arclen * (t12 / len_t12) # straight hole degenerate case

### Calculate survey

In [3]:
P1 = np.array([40.00, 40.00, 700.00]) # position at start (tie-in)
print ('P1:\t', P1)
srv = np.array([[702.55, 5.50, 45.00], [1964.57, 29.75, 77.05], [5086.35, 29.75, 77.05], [9901.68, 120.00, 285.00]])
print ('srv:', srv)

P1:	 [ 40.00  40.00  700.00]
srv: [[ 702.55  5.50  45.00]
 [ 1964.57  29.75  77.05]
 [ 5086.35  29.75  77.05]
 [ 9901.68  120.00  285.00]]


In [4]:
dir = np.array([toUnitDir(inc, az) for inc, az in srv[:,1:3]])
print ('dir:', dir)

dir: [[ 0.07  0.07  1.00]
 [ 0.11  0.48  0.87]
 [ 0.11  0.48  0.87]
 [ 0.22 -0.84 -0.50]]


In [8]:
par = list(zip(dir[:-1], dir[1:]))
print ('par:', par)
delta_md = srv[1:,0] - srv[:-1,0]
print ('delta_md:', delta_md)
ang = np.array([angle(p[0], p[1]) for p in par])
print ('ang:', ang)
rad = delta_md / ang
print ('rad:', rad)

par: [(array([ 0.07,  0.07,  1.00]), array([ 0.11,  0.48,  0.87])), (array([ 0.11,  0.48,  0.87]), array([ 0.11,  0.48,  0.87])), (array([ 0.11,  0.48,  0.87]), array([ 0.22, -0.84, -0.50]))]
delta_md: [ 1262.02  3121.78  4815.33]
ang: [ 0.44  0.00  2.52]
rad: [ 2864.58  inf  1909.86]


<ipython-input-8-a6291afbcd0e>:7: RuntimeWarning: divide by zero encountered in true_divide
  rad = delta_md / ang


In [9]:
rel = np.array([np.array([0.0, 0.0, 0.0])] + [arc2chord(t[0], t[1], arclen) for t, arclen in zip(par, delta_md)])
print ('rel:', rel)
pos = np.cumsum(rel, axis=0) + P1
print ('pos:', pos)

rel: [[ 0.00  0.00  0.00]
 [ 114.80  353.66  1195.34]
 [ 347.15  1509.68  2710.33]
 [ 1998.41 -2103.13  2194.18]]
pos: [[ 40.00  40.00  700.00]
 [ 154.80  393.66  1895.34]
 [ 501.95  1903.34  4605.67]
 [ 2500.35 -199.80  6799.85]]


### Build and Hold

In [10]:
P1 = pos[0] # position at start
s1 = srv[0] # md, inc, az at start
t1 = toUnitDir(s1[INC], s1[AZ]) # unit direction at start
R = rad[0] # radius of curvature
P3 = pos[2] # position at end
md2 = srv[1][0]
md3 = srv[2][0]
print ('beta:', md3 - md2)

beta: 3121.7800000000007


In [11]:
P3_P1 = P3 - P1
print ('P3_P1:\t', P3_P1)
psi = np.linalg.norm(P3_P1)
print ('psi:\t', psi)
if psi**2 + 1.0 == 1.0:
    raise Exception('The start and end points are not disjoint!')

P3_P1:	 [ 461.95  1863.34  3905.67]
psi:	 4351.97399466106


In [12]:
eta = np.dot(t1, P3_P1)
print ('eta:\t', eta)
xi = np.sqrt(psi**2 - eta**2)
print ('xi:\t', xi)
rho = xi - R
print ('rho:\t', rho)
omaga = np.sqrt(eta**2 + rho**2)
print ('omaga\t', omaga)
beta = np.sqrt(omaga**2 - R**2)
print ('beta:\t', beta)

eta:	 4045.280646659934
xi:	 1604.7997195802122
rho:	 -1259.7843059613579
omaga	 4236.903587266053
beta:	 3121.7800000000007


In [ ]:
if eta <= 0.0 and xi <= 2.0 * R:
    raise Exception('alpha is greater than or equal to 2 PI!')

In [ ]:
P4 = P1 + beta * -t1
te = (P3 - P4) / np.linalg.norm(P3 - P4)
t2 = reflect(te, t1)
P2 = P3 + beta * -t2
alpha = angle(t1, t2)
s12 = R * alpha

In [ ]:
print ('P2:\t', P2)
print ('pos[1]\t', pos[1])
s2 = np.array([s1[MD] + s12] + toSpherical(t2).tolist())
print ('s2:\t', s2)
print ('srv[1]\t', srv[1])

In [ ]:
if np.linalg.norm(P3 - P2)**2 + 1.0 == 1.0:
    raise Exception('The mid and end points are not disjoint!')
if R**2 >= psi**2 + eta**2:
    print ('Radius is greater than critical...')

In [ ]:
# if radius is greater than critical...
te = P3_P1 / psi
t2 = reflect(te, t1)
P2 = P3
R = psi**2 / 2.0 * xi
alpha = angle(t1, t2)
s12 = R * alpha

In [ ]:
print ('P2:\t', P2)
s2 = np.array([s1[MD] + s12] + toSpherical(t2).tolist())
print ('s2:\t', s2)
print ('R:\t', R)

### Hold and Build

In [ ]:
P1 = pos[1] # position at start
s1 = srv[1] # md, inc, az at start
t1 = toUnitDir(s1[INC], s1[AZ]) # unit direction at start (reverse because we are going in the other direction)
R = rad[2] # radius of curvature
P3 = pos[3] # position at end

In [ ]:
P3_P1 = P3 - P1
print ('P3_P1:\t', P3_P1)
psi = np.linalg.norm(P3_P1)
print ('psi:\t', psi)
eta = np.dot(t1, P3_P1)
print ('eta:\t', eta)
xi = np.sqrt(psi**2 - eta**2)
print ('xi:\t', xi)
rho = xi - R
print ('rho:\t', rho)
gamma = np.sqrt(R**2 - rho**2)
print ('gamma:\t', gamma)
beta = eta - gamma
print ('beta:\t', beta)

In [ ]:
if eta <= 0.0 and xi <= 2.0 * R:
    raise Exception('alpha is greater than or equal to 2 PI!')

In [ ]:
P2 = P1 + beta * t1
te = (P3 - P2) / np.linalg.norm(P3 - P2)
t2 = t1
t3 = reflect(te, t2)
alpha = angle(t2, t3)
s12 = R * alpha

In [ ]:
print ('P2:\t', P2)
print ('pos[2]\t', pos[2])
s2 = np.array([s1[MD] + beta] + toSpherical(t2).tolist())
print ('s2:\t', s2)
print ('srv[2]\t', srv[2])
print ('P3:\t', P3)
print ('pos[3]\t', pos[3])
s3 = np.array([s2[MD] + s12] + toSpherical(t3).tolist())
print ('s3:\t', s3)
print ('srv[3]\t', srv[3])

In [ ]:
if R**2 >= rho**2 + eta**2:
    print 'Radius is greater than critical...'

In [ ]:
te = P3_P1 / psi
t3 = reflect(te, t1)
alpha = angle(t1, t3)
R = psi**2 / 2.0 * xi
s12 = R * alpha

In [ ]:
print ('P3:\t', P3)
s3 = np.array([s1[MD] + s12] + toSpherical(t3).tolist())
print ('s3:\t', s3)
print ('R:\t', R)